Why we use `Naive bayes` ?

**Step 1** - We are trying to calculate what is probability of class 1 (other will be just 1- p(1)) given a document x. i.e. P(c=1|x).  
**Step 2** - Using naive bayes formula, we can rewrite it as: P(c=1|x) = {P(x|c=1) P(c=1)} / P(x) . Why we did this transformation? Because from data we know that how many of total docs are class = 1, and how many of class 1 have words that we have in our document. We can rewrite -> P(x|c=1) as P(x1|c=1) P(x2|c=1) P(x3|c=1) if x1,x2,x3 are 3 words in our doc x (considering independence of x1,x2,x3). But we can not rewrite, P(c=1|x) as P(c=1|x1) P(c=1|x2) P(c=1|x3). So just a mathematical transformaiton to easily calculate probabilities.   

**Step 3** So from data (term doc matrix), what we need are no. of times class = 1, no. of times some word appear whereever class = 1. (same with class 0 too)

In [65]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ParameterGrid
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)


from fastai.nlp import *
from sklearn.linear_model import LogisticRegression
from torchtext import vocab, data, datasets
from IPython.core.display import Image 

In [61]:
#! ls /Users/groverprince/Documents/msan/msan_ml/data/aclImdb/

The sentiment classification task consists of predicting the polarity (positive or negative) of a given text.
To get the dataset, in your terminal run the following commands:  
`wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz`   
`gunzip aclImdb_v1.tar.gz`  
`tar -xvf aclImdb_v1.tar`  

In [4]:
PATH='/Users/groverprince/Documents/msan/msan_ml/data/aclImdb/'
names = ['neg','pos']

In [66]:
desk = '/Users/groverprince/Desktop/'

In [5]:
trn, trn_Y = texts_from_folders(f'{PATH}train', names) # taking text from each file, saving in list. saving 0 or 1 in other list

In [6]:
val,val_y = texts_from_folders(f'{PATH}test',names)

In [7]:
trn[0], trn_Y[0], len(trn)

("Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly.",
 0,
 25000)

In [8]:
val[0],val_y[0], len(val)

("Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in.",
 0,
 25000)

In [9]:
# this is just like "this is good" --> this = 1, is = 1, good = 1
# but it is just an object

veczr = CountVectorizer(tokenizer=tokenize) # making tokenizer (bag of words representation) - sparse matrix

In [10]:
# using veczr object to make term doc matrix

trn_term_doc = veczr.fit_transform(trn)  
val_term_doc = veczr.transform(val)

In [11]:
len(set(val[0].split()))

128

In [12]:
trn_term_doc[0], val_term_doc

(<1x75132 sparse matrix of type '<class 'numpy.int64'>'
 	with 93 stored elements in Compressed Sparse Row format>,
 <25000x75132 sparse matrix of type '<class 'numpy.int64'>'
 	with 3640465 stored elements in Compressed Sparse Row format>)

* 25000 = number of training documents . 
* 75132 = number of unique words (represent columns of tokenized version) in all documents combined . keys of dict
* training term document has 93 words out of those 75132 unique words. 
* 128 should technically have been 93. maybe because of ./', signs

In [13]:
100000*(146**-0.75)*(np.log10(1+np.log10(162)))

1205.7655009765356

to get `feature_names` in particular column index

In [14]:
vocab = veczr.get_feature_names(); vocab[3000:3005]

['ameteurish', 'ametuer', 'amfortas', 'amg', 'ami']

opposite of above, to get index of particular `column name (vocublary)`. _ is just used to replace inplace (not here) what is it for ?

In [15]:
veczr.vocabulary_['ametuer']

3001

what is going on here?

In [16]:
trn_term_doc[0,2297]

0

### Use of Naive Bayes to fit a model

In [17]:
x=trn_term_doc
y=trn_Y

In [18]:
x.shape # 75312 columns, distinct words, 25000 documents/rows

(25000, 75132)

In [19]:
p = x[y==1].sum(0)+1  # no of times a word appear, given class = 1. 
q = x[y==0].sum(0)+1  # no of times a word appear, given class = 0

In [20]:
p.shape, q.shape

((1, 75132), (1, 75132))

In [21]:
p, q

(matrix([[    2,     1, 11820, ...,     2,     2,     1]], dtype=int64),
 matrix([[    1,     2, 12742, ...,     1,     2,     8]], dtype=int64))

p/psum =  (total times word a1 appear in all docs)/(total words appearning in all the docs) (+ 1 is added for each word, ignoring that for understanding for now). (all this given class = 1)
What does it mean? It means, given class = 1, what is probability of word a1 appearing. 

we calculate ratio of this wrt when class = 0 as ratio gives whether this word has higher chance of being from class 1 or 0.

In [22]:
q.sum(1) , p.sum(1) # total words in 0 class are more

(matrix([[3747495]], dtype=int64), matrix([[3789022]], dtype=int64))

In [23]:
r = np.log((p/p.sum(1))/(q/q.sum(1))) 
r.shape, r

((1, 75132),
 matrix([[ 0.68213, -0.70417, -0.08613, ...,  0.68213, -0.01102, -2.09046]]))

In [30]:
b = np.log(p/q)
b.shape, b

((1, 75132),
 matrix([[ 0.69315, -0.69315, -0.07511, ...,  0.69315,  0.     , -2.07944]]))

by using naive bayes prediction, we are sort of directly making a model and those predictions can help us to predict

In [31]:
r.T.shape

(75132, 1)

In [32]:
val_term_doc

<25000x75132 sparse matrix of type '<class 'numpy.int64'>'
	with 3640465 stored elements in Compressed Sparse Row format>

validation has 1 or no. of times occurence of word in doc where that word appears, 0 otherwise. so it will multiply `word count in doc with chances of word appearning given class 1 probability is higher` 

In [33]:
pre_preds = val_term_doc @ r.T 

In [34]:
pre_preds.T

matrix([[ -8.33253, -22.4765 , -26.97431, ...,  97.91732,  18.31415,  21.76217]])

In [36]:
pre_preds = val_term_doc @ r.T
preds = pre_preds.T>0  # because if sum of log ratios is > 0 means class 1
(preds==val_y).mean()

0.80740000000000001

binarised naive bayes. Funda is don't need to take number of occureces of word in a document. Just whether is appears or not is fine. Think of a case where a word appears many many times but only in 1 document for class 1 and once in many documents of class 0. Where should it be assigned. Class 1 or 0? Here it is trying to assign it to class 0 which makes more sense and that appearance in only 1 doc for class 1 might just be an outlier

In [29]:
pre_preds = val_term_doc.sign() @ r.T + b
preds = pre_preds.T>0
(preds==val_y).mean()

0.82623999999999997

In [30]:
pre_preds.T

matrix([[ -4.35523, -19.08058, -23.69877, ...,  54.90807,  14.71441,  18.40103]])

In [31]:
r

matrix([[ 0.68213, -0.70417, -0.08613, ...,  0.68213, -0.01102, -2.09046]])

Now let's try same thing using **Logistic regression**. (Considering onyl unigrams like this first)

`C` in Logistic is inverse of regularization constant ($ \lambda $). So lower c means high regularization. Let's try first without any regularization. Then we will move on to regularization to see how it improves (by regularizing parameters we aim to hangle overfitting)

In [32]:
m = LogisticRegression(C = 1e5)
m.fit(x,y)  # feeding raw form of sparse matrix and target y without changin anything -- so easy :D
preds = m.predict(val_term_doc)
np.mean(preds == val_y)

0.85387999999999997

One thing that we tried in NBC, we can try here too. Ignore no. of times a word appears in document. Even if it word `a1` appears 5 times in a doc, just take 1. That what `val_term_doc.sign()` can do. It return 1 for +ve, 0 for 0 and -1 for all negative values. 

In [33]:
m = LogisticRegression(C = 1e5)
m.fit(x.sign(),y)  # sign() in training data
preds = m.predict(val_term_doc.sign())  # sign in test data
np.mean(preds == val_y)

0.85768

So, that's observable! binarising our matrix values improves accuracy. .85768 > .85387

Now let's try some regularization. (maybe we are overfitting). But let's first check if we are actually overfitting or not. Way to do that is to check both training and validation score. If training score is very high, then there is chance of overfitting. Let's check that

In [34]:
preds = m.predict(x.sign())
np.mean(preds == y)

1.0

Wow. training accuracy is 100%. Definitely it is overfitting. Let's reduce it by introducing some regularization. 
Starting with `c = 0.1` first

In [35]:
m = LogisticRegression(C = 0.1)
m.fit(x.sign(),y)  # sign() in training data
preds = m.predict(val_term_doc.sign())  # sign in test data
preds2 = m.predict(x.sign())

np.mean(preds == y), np.mean(preds2 == y)

(0.88400000000000001, 0.96736)

Ok. So our validation accuracy has definitely improved using some regularization and training has reduced too. Let's try to tune `C` to find best value of it by using a gridsearch

In [36]:
def fitOne(model, X, y, params):
    m = model(**params)
    return m.fit(X, y)

def fitModels(model, paramGrid, X, y, n_jobs=-1, verbose=10):
    return Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(fitOne)(model,
                                                                    X,
                                                                    y,
                                                                    params) for params in paramGrid)

In [51]:
model = LogisticRegression
grid = {
    'C': [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100], # regularization
    'penalty': ['l2'], # penalty type
    'n_jobs': [-1] # parallelize within each fit over all cores
}
paramGrid = ParameterGrid(grid)
myModels = fitModels(model, paramGrid, x.sign(), y)

[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done   2 out of   7 | elapsed:    1.9s remaining:    4.8s
[Parallel(n_jobs=-1)]: Done   3 out of   7 | elapsed:    2.8s remaining:    3.8s
[Parallel(n_jobs=-1)]: Done   4 out of   7 | elapsed:    4.6s remaining:    3.5s
[Parallel(n_jobs=-1)]: Done   5 out of   7 | elapsed:    7.7s remaining:    3.1s
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:   10.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:   10.0s finished


In [52]:
def print_score(m):
    preds = m.predict(val_term_doc.sign())  # sign in test data
    preds2 = m.predict(x.sign())

    print(np.mean(preds == y), np.mean(preds2 == y))

In [53]:
for i in myModels:
    print_score(i)

0.80576 0.81452
0.85056 0.8644
0.8798 0.91468
0.884 0.96736
0.87384 0.99784
0.86824 1.0
0.8638 1.0


In above print, lhs is validation score and rhs is train score. Validation score is highest for `C` = 0.1 and we can chose 0.1 as our regularization 

What else can be tried to improve the accuracy? One thing that can be tried is tweaking Naive bayes a little bit. We assumed our words/features to be independent of each other. Let's try some pair interaction that is something like making new features which are made using combinations of old independent features.  

We can use `bigrams` or `trigrams`. What are `bigrams`. Very simple concept. Some words tend to appear together very frequently. Like `this is`, `what are`, `you are` etc. One major drawback of `naive` bayes (that's why it is called naive :P) is that it assumes all our features to be independent. But in actual sense they are not. Like frequenct occuring `what are`. these will come together and be dependent of each other. 

Bigram example -- `what are you doing` -> `what are`, `are you`, `you doing` . Adding bigrams will add both sinle word double adjacent word features

In [37]:
# ngram_range=(1,2) does unigram, bigram and trigram. 
# max_features is limiting documents to not make each and every combination of adjacent 3 features. 
# It first sorts tri/bi grams based on number of together occurences and only takes ngrams which appear most.

veczr =  CountVectorizer(ngram_range=(1,3), tokenizer=tokenize, max_features=800000)
trn_term_doc = veczr.fit_transform(trn)
val_term_doc = veczr.transform(val)

In [38]:
trn_term_doc.shape, val_term_doc.shape # 25k rows and max_feature (800k) columns

((25000, 800000), (25000, 800000))

let's see how vocublary look like now

In [39]:
vocab = veczr.get_feature_names()

In [40]:
vocab[1002:1010]

['! the camera',
 '! the cast',
 '! the character',
 '! the characters',
 '! the cinematography',
 '! the climax',
 '! the dialog',
 '! the direction']

these words tend to appear together a lot.

In [41]:
# repeating same steps to extract x, y 

x = trn_term_doc.sign()
y = trn_Y
val_x = val_term_doc.sign()

p = x[y == 1].sum(0) + 1  # i need sum of words appearing in class 1. i.e in all docs
q = x[y == 0].sum(0) + 1
r = np.log((p/p.sum()) / (q/q.sum())) # r is log count ratio. log of count ratios
b = np.log((y==1).sum()/(y==0).sum())

In [42]:
# naive bayes again

pre_preds = val_term_doc.sign() @ r.T
preds = pre_preds.T>0
(preds==val_y).mean()

0.88044

In [74]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(x, y);

preds = m.predict(val_x)
(preds.T==val_y).mean()

0.90500000000000003

Now the idea is to fit a logistic regression model with updated input matrix. We are updating input x to x.mult.r where r is log count ratio. Why? So, in NB solution x@r.T was our output (matrix mult) which was saying that `r` is a good weight matrix for our linear layer. Now

In [80]:
x.multiply(r)

<25000x800000 sparse matrix of type '<class 'numpy.float64'>'
	with 12589101 stored elements in COOrdinate format>

In [84]:
x@r.T

matrix([[ -19.31273],
        [-322.31574],
        [ -36.51194],
        ..., 
        [ 101.11562],
        [  99.71359],
        [  42.95914]])

In [85]:
x_nb = x.multiply(r)

m = LogisticRegression(dual=True, C=0.1)
m.fit(x_nb, y);

val_x_nb = val_x.multiply(r)

preds = m.predict(val_x_nb)
(preds.T==val_y).mean()

0.91768000000000005

Accuracy improved from the previous version

### fasti NB SVM

In [43]:
si = 2000 #(stop iteration)

In [47]:
??TextClassifierData

In [53]:
??dotprod_nb_learner() # gets model . dot product with naive bayes like in above case 

In [46]:
# Here is how we get a model from a bag of words
md = TextClassifierData.from_bow(trn_term_doc, trn_Y, val_term_doc, val_y, si)

In [54]:
learner = md.dotprod_nb_learner()  # 
learner.fit(0.02, 1, wds=1e-6, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       0.02432  0.11949  0.91768]                         



In [55]:
learner.fit(0.02, 2, wds=1e-6, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

[ 0.       0.01933  0.11368  0.92148]                         
[ 1.       0.01358  0.1118   0.92104]                          



** 92.1% accurately predict whether IMDB review is good or bad**